In [1]:
import numpy as np
import itertools
import cv2
import random
import copy
from time import sleep, time
import pickle
import concurrent.futures
import os
import pandas as pd

In [2]:
import USDMM_v3 as USDMM

In [449]:
from importlib import reload
reload(USDMM)


<module 'USDMM_v3' from '/Users/mariana.feldman/Documents/GitHub/USDMM/USDMM_v3.py'>

In [450]:


WOMC = USDMM.WOMC(
    new = True, # True/ STR -> If True inicialize with the cross operator/ If STR it will opens the file with the name passed
    nlayer = 2, # INT -> Number of operators in each layer
    wlen = 3,  # INT -> Size of the operator (wlen*wlen)
    train_size = 10, # INT -> Number of images to train 
    val_size = 10, # INT -> Number of images to validate
    test_size = 10, # INT -> Number of images to test
    error_type = 'iou', # 'mae' / 'iou' -> type of error
    neighbors_sample = False, # INT/False -> Number of neighbors to sort
    epoch_f = 10, # INT -> Number of epochs for the boolean function lattice (fixed windows)
    epoch_w = 50, # INT -> Number of epochs for the windows lattice
    batch = 10, # INT -> Batch size
    path_results = 'results_V2', # STR -> file where we want to save the results
    name_save='_V2', # STR -> pos fixed name for the results saved
    seed = 0, #INT -> seed for reproducibilit
    parallel = True, # True/False -> use parallel (True) or sequential (False)
    early_stop_round_f = 20, #INT -> max number of epochs without changes in the boolean function lattice
    early_stop_round_w = 10 #INT -> max number of epochs without changes in the windows lattice
)

Janela Inicializada: [array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan]), array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]
Resultados serão salvos em  results_V2
------------------------------------------------------------------


In [452]:
X1, y1 = WOMC.get_images(WOMC.train_size, 'train')

In [453]:
X2, y2, x_shape = WOMC.get_images_2(WOMC.train_size, 'train')

In [454]:
X1 = np.array(X1)
y1 = np.array(y1)

In [455]:
X1.shape, X2.shape

((10, 56, 56), (9, 31360))

In [13]:
error = 0
n_samples = len(y1)
for k in range(n_samples):
    y_z = copy.deepcopy(y1[k])
    y_z[y_z==-1]=0
    union = np.sum(np.maximum(X1,y_z)==1)
    interc = np.sum(X1 +y_z == 2)
    error += (1 - interc/union)
error = error/n_samples
error

0.8829005589351997

In [15]:
middle_row_index = X2.shape[0] // 2
X22 = X2[middle_row_index].reshape(1, -1)
X22.shape

(1, 31360)

In [16]:
WOMC.compute_cost(X22, y2)

0.8829005589351997

In [741]:
WOMC.W

[array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan]),
 array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]

In [18]:
WW = [np.array([np.nan,  np.nan, np.nan,  np.nan,  1.,  1., np.nan,  np.nan, np.nan]),np.array([np.nan,  np.nan, np.nan,  np.nan,  1.,  1., np.nan,  np.nan, np.nan])]

In [263]:
WW = [np.array([np.nan,  1., np.nan,  np.nan,  1.,  1, np.nan,  np.nan, np.nan]),np.array([np.nan,  1., np.nan,  np.nan,  1.,  1, np.nan,  np.nan, np.nan])]

In [270]:
WW

[array([nan,  1., nan, nan,  1.,  1., nan, nan, nan]),
 array([nan,  1., nan, nan,  1.,  1., nan, nan, nan])]

In [456]:
nn_parameters = WOMC.initialize_parameters_nn(WOMC.W)

In [444]:
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0., -1., -1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1.,  1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.,  1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1., -1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1., -1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1., -1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.,  1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1.,  1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.,  1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.],
         [ 0.,  1.,  0., -1., -1., -1.,  0., -1.,  0.],
         [ 0.,  1.,  0., -1., -1., -

In [436]:
nn_parameters['l0']['W1'] = np.array([[1, 1, 1, 1, 0, 0, 1, 1]])
nn_parameters['l1']['W1'] = np.array([[1, 1, 1, 1, 0, 0, 1, 1]])
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
  'b0': array([[-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2]]),
  'W1': array([[1, 1, 1, 1, 0, 0, 1, 1]]),
  'b1': array([[0.]])},
 'l1': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  

In [265]:
nn_parameters = WOMC.initialize_parameters_nn(WW)
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
  'b0': array([[-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2]]),
  'W1': array([[1, 1, 0, 0, 1, 1, 1, 1]]),
  'b1': array([[0.]])},
 'l1': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  

In [242]:
nn_parameters['l0']['W1'] = np.array([[1, 0, 1, 0]])
nn_parameters['l1']['W1'] = np.array([[1, 0, 1, 0]])
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]]),
  'b0': array([[-1],
         [-1],
         [-1],
         [-1]]),
  'W1': array([[1, 0, 1, 0]]),
  'b1': array([[0.]])},
 'l1': {'W0': array([[ 0., -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]]),
  'b0': array([[-1],
         [-1],
         [-1],
         [-1]]),
  'W1': array([[1, 0, 1, 0]]),
  'b1': array([[0.]])}}

In [437]:
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
  'b0': array([[-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2]]),
  'W1': array([[1, 1, 1, 1, 0, 0, 1, 1]]),
  'b1': array([[0.]])},
 'l1': {'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
         [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  

In [400]:
AL, caches = WOMC.L_model_forward(X2, nn_parameters)

In [401]:
np.unique(AL)

array([0, 1])

In [402]:
AL.shape

(1, 31360)

In [403]:
WOMC.compute_cost(AL, y2)

0.9488385468786822

In [404]:
ALL = np.where(AL <= 0, -1, 1)

In [405]:
a_img = WOMC.transform_nn_for_image(ALL, WOMC.train_size)
a_img.shape

(10, 56, 56)

In [382]:
Z = np.dot( nn_parameters['l0']["W0"], X2) + nn_parameters['l0']["b0"]
Z

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-3., -3., -3., ..., -3., -3., -3.],
       [-3., -3., -3., ..., -3., -3., -3.],
       [-5., -5., -5., ..., -5., -5., -5.]])

In [383]:
Z ,_= WOMC.L_forward(X2, nn_parameters['l0']["W0"], nn_parameters['l0']["b0"])
Z

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-3., -3., -3., ..., -3., -3., -3.],
       [-3., -3., -3., ..., -3., -3., -3.],
       [-5., -5., -5., ..., -5., -5., -5.]])

In [384]:
Z = np.dot( nn_parameters['l0']["W0"], X2) + nn_parameters['l0']["b0"]
Az = np.where(Z < 0, 0, 1)

In [385]:
Az = np.where(Z < 0, 0, 1)
Az

array([[1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [386]:
elementos_unicos, contagens = np.unique(Az, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: 0, Contagem: 219520
Elemento: 1, Contagem: 31360


In [387]:
np.unique(Az)

array([0, 1])

In [390]:
Zz ,_= WOMC.L_forward(Az, nn_parameters['l0']["W1"], nn_parameters['l0']["b1"])
Azz = np.where(Zz <= 0, 0, 1)

In [391]:
elementos_unicos, contagens = np.unique(Azz, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: 0, Contagem: 766
Elemento: 1, Contagem: 30594


In [350]:
A, cache = WOMC.L_activation_forward(
        X2, nn_parameters['l0']["W0"], nn_parameters['l0']["b0"])
A

array([[1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [215]:
AAA = WOMC.L_activation_forward(
        A, nn_parameters['l0']["W1"], nn_parameters['l0']["b1"])
AAA

(array([[-1, -1, -1, ..., -1, -1, -1]]),
 ((array([[ 1,  1,  1, ...,  1,  1,  1],
          [-1, -1, -1, ..., -1, -1, -1],
          [-1, -1, -1, ..., -1, -1, -1],
          ...,
          [-1, -1, -1, ..., -1, -1, -1],
          [-1, -1, -1, ..., -1, -1, -1],
          [-1, -1, -1, ..., -1, -1, -1]]),
   array([0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
          0, 1, 0, 1, 1, 1, 0, 1, 0, 0]),
   array([[0.]])),
  array([[-18., -18., -18., ..., -18., -18., -18.]])))

In [351]:
A = X2
for l in range(2):
    A_prev = A
    A, cache = WOMC.L_activation_forward(
        A_prev, nn_parameters['l0']["W" + str(l)], nn_parameters['l0']["b" + str(l)])


In [352]:
np.unique(A)

array([1])

In [291]:
elementos_unicos, contagens = np.unique(A, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: -1, Contagem: 31360


In [48]:
A.shape

(1, 31360)

In [106]:
a_img = WOMC.transform_nn_for_image(A, WOMC.train_size)
a_img.shape

(10, 56, 56)

In [85]:
A2 = WOMC.transform_image_forlayer(A, WOMC.train_size)
A2.shape

(9, 31360)

In [102]:
X22.shape

(1, 31360)

In [104]:
x_img = WOMC.transform_image_forlayer(X22, WOMC.train_size)
x_img.shape

(9, 31360)

In [105]:
np.array_equal(x_img, X2)

True

In [59]:
nn_parameters['l0']

{'W0': array([[ 0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
 'b0': array([[-1],
        [-1],
        [-1],
        [-1]]),
 'W1': array([[1, 0, 0, 1]]),
 'b1': array([[0.]])}

In [58]:
nn_parameters['l1']

{'W0': array([[ 0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
 'b0': array([[-1],
        [-1],
        [-1],
        [-1]]),
 'W1': array([[1, 0, 0, 1]]),
 'b1': array([[0.]])}

In [52]:
Al = A2
for l in range(2):
    A_prev = Al
    Al, cache = WOMC.L_activation_forward(
        A_prev, nn_parameters['l1']["W" + str(l)], nn_parameters['l1']["b" + str(l)])

In [53]:
np.unique(Al)

array([-1,  1])

In [123]:
np.array_equal(Al, AL)

True

In [124]:
a_img = WOMC.transform_nn_for_image(Al, WOMC.train_size)
a_img.shape

(10, 56, 56)

In [721]:
AL, caches = WOMC.L_model_forward(X2, nn_parameters)

In [722]:
np.unique(AL)

array([-1])

In [677]:
A

array([[1, 1, 1, ..., 1, 1, 1]])

In [680]:
A.shape

(1, 31360)

In [871]:
a_img = WOMC.transform_nn_for_image(A, WOMC.train_size)
a_img.shape

(10, 56, 56)

In [733]:
error = 0
n_samples = len(y1)
for k in range(n_samples):
    y_z = copy.deepcopy(y1[k])
    y_z[y_z==-1]=0
    union = np.sum(np.maximum(a_img,y_z)==1)
    interc = np.sum(a_img +y_z == 2)
    error += (1 - interc/union)
error = error/n_samples
error

0.9559336260801687

In [165]:
reload(USDMM_2)


<module 'USDMM_v2' from '/Users/mariana.feldman/Documents/GitHub/USDMM/USDMM_v2.py'>

In [166]:
import USDMM_v2 as USDMM_2


WOMC2 = USDMM_2.WOMC(
    new = True, # True/ STR -> If True inicialize with the cross operator/ If STR it will opens the file with the name passed
    nlayer = 2, # INT -> Number of operators in each layer
    wlen = 3,  # INT -> Size of the operator (wlen*wlen)
    train_size = 10, # INT -> Number of images to train 
    val_size = 10, # INT -> Number of images to validate
    test_size = 10, # INT -> Number of images to test
    error_type = 'iou', # 'mae' / 'iou' -> type of error
    neighbors_sample = False, # INT/False -> Number of neighbors to sort
    epoch_f = 10, # INT -> Number of epochs for the boolean function lattice (fixed windows)
    epoch_w = 50, # INT -> Number of epochs for the windows lattice
    batch = 10, # INT -> Batch size
    path_results = 'results_V2', # STR -> file where we want to save the results
    name_save='_V2', # STR -> pos fixed name for the results saved
    seed = 0, #INT -> seed for reproducibilit
    parallel = True, # True/False -> use parallel (True) or sequential (False)
    early_stop_round_f = 20, #INT -> max number of epochs without changes in the boolean function lattice
    early_stop_round_w = 10 #INT -> max number of epochs without changes in the windows lattice
)

Janela Inicializada: [array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan]), array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]
Joint Inicializada: [array([['00000', '1'],
       ['00001', '0'],
       ['00010', '0'],
       ['00011', '1'],
       ['00100', '0'],
       ['00101', '1'],
       ['00110', '1'],
       ['00111', '1'],
       ['01000', '1'],
       ['01001', '1'],
       ['01010', '1'],
       ['01011', '0'],
       ['01100', '1'],
       ['01101', '0'],
       ['01110', '0'],
       ['01111', '1'],
       ['10000', '0'],
       ['10001', '0'],
       ['10010', '1'],
       ['10011', '0'],
       ['10100', '1'],
       ['10101', '0'],
       ['10110', '1'],
       ['10111', '1'],
       ['11000', '0'],
       ['11001', '0'],
       ['11010', '0'],
       ['11011', '1'],
       ['11100', '0'],
       ['11101', '0'],
       ['11110', '0'],
       ['11111', '0']], dtype='<U21'), array([['00000', '0'],
       ['00001', '1'],
       ['00010', '0'],
       ['00011', '0'],
     

In [460]:
(joint, error, epoch_min) = WOMC2.get_error_window(WOMC2.W, WOMC2.joint, 0)

In [461]:
error

array([0.61747601, 0.61736838])

In [271]:
WW

[array([nan,  1., nan, nan,  1.,  1., nan, nan, nan]),
 array([nan,  1., nan, nan,  1.,  1., nan, nan, nan])]

In [272]:
joint = WOMC2.create_joint(WW[0])
joint

array([['000', '1'],
       ['001', '1'],
       ['010', '1'],
       ['011', '1'],
       ['100', '0'],
       ['101', '0'],
       ['110', '1'],
       ['111', '1']], dtype='<U21')

In [273]:
joint = [joint, joint]

In [274]:
joint

[array([['000', '1'],
        ['001', '1'],
        ['010', '1'],
        ['011', '1'],
        ['100', '0'],
        ['101', '0'],
        ['110', '1'],
        ['111', '1']], dtype='<U21'),
 array([['000', '1'],
        ['001', '1'],
        ['010', '1'],
        ['011', '1'],
        ['100', '0'],
        ['101', '0'],
        ['110', '1'],
        ['111', '1']], dtype='<U21')]

In [184]:
#joint = WOMC2.create_joint(WOMC2.W[0])
joint

array([['00000', '0'],
       ['00001', '1'],
       ['00010', '1'],
       ['00011', '0'],
       ['00100', '1'],
       ['00101', '0'],
       ['00110', '0'],
       ['00111', '1'],
       ['01000', '1'],
       ['01001', '1'],
       ['01010', '1'],
       ['01011', '0'],
       ['01100', '0'],
       ['01101', '1'],
       ['01110', '1'],
       ['01111', '1'],
       ['10000', '0'],
       ['10001', '0'],
       ['10010', '1'],
       ['10011', '0'],
       ['10100', '1'],
       ['10101', '1'],
       ['10110', '0'],
       ['10111', '1'],
       ['11000', '0'],
       ['11001', '1'],
       ['11010', '1'],
       ['11011', '1'],
       ['11100', '0'],
       ['11101', '1'],
       ['11110', '0'],
       ['11111', '0']], dtype='<U21')

In [854]:
joint[1][1] = '1'
joint[1][1] = '1'
joint[2][1] = '0'
joint[3][1] = '0'
joint[4][1] = '1'
joint[5][1] = '1'
joint[6][1] = '1'
joint[7][1] = '1'
joint[8][1] = '1'
joint[9][1] = '0'
joint[10][1] = '0'
joint[11][1] = '1'
joint[12][1] = '0'
joint[13][1] = '1'
joint[14][1] = '1'
joint[15][1] = '0'
joint[16][1] = '0'
joint[17][1] = '1'
joint[18][1] = '0'
joint[19][1] = '0'
joint[20][1] = '0'
joint[21][1] = '1'
joint[22][1] = '0'
joint[23][1] = '0'
joint[24][1] = '1'
joint[25][1] = '0'
joint[26][1] = '0'
joint[27][1] = '0'
joint[28][1] = '1'
joint[29][1] = '0'
joint[30][1] = '0'
joint[31][1] = '0'

In [185]:
joint = [joint, joint]

In [275]:
joint[0][:,1].astype(int)

array([1, 1, 1, 1, 0, 0, 1, 1])

In [292]:
W_matrices = WOMC2.create_w_matrices(WW,joint)
#W_matrices = WOMC2.create_w_matrices(WOMC2.W,joint)

In [302]:
W_matrices[0]

[array([[ 0., -1.,  0.],
        [ 0., -1., -1.],
        [ 0.,  0.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 0., -1.,  1.],
        [ 0.,  0.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 0.,  1., -1.],
        [ 0.,  0.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 0.,  1.,  1.],
        [ 0.,  0.,  0.]]),
 array([[ 0.,  1.,  0.],
        [ 0.,  1., -1.],
        [ 0.,  0.,  0.]]),
 array([[0., 1., 0.],
        [0., 1., 1.],
        [0., 0., 0.]])]

In [357]:
WOMC2.train[0][6:9, 3:6]

array([[-1.,  1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.]])

In [359]:
elementos_unicos, contagens = np.unique(X2, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")


Elemento: -1.0, Contagem: 229650
Elemento: 1.0, Contagem: 52590


In [360]:
np.where(X2 == 1)

(array([0, 0, 0, ..., 8, 8, 8]),
 array([  265,   397,   497, ..., 31090, 31118, 31129]))

In [366]:
X2[:,264]

array([-1.,  1., -1., -1., -1., -1., -1., -1., -1.])

In [367]:
np.dot(nn_parameters['l0']['W0'], X2)[:,264]

array([ 1., -1., -1., -3.,  3.,  1.,  1., -1.])

In [368]:
(np.dot(nn_parameters['l0']['W0'], X2) + nn_parameters['l0']['b0']).shape

(8, 31360)

In [369]:
Z = np.dot(nn_parameters['l0']['W0'], X2) + nn_parameters['l0']['b0']
Z[:,264]

array([-1., -3., -3., -5.,  1., -1., -1., -3.])

In [370]:
A = np.where(Z < 0, 0, 1)
A[:,264]

array([0, 0, 0, 0, 1, 0, 0, 0])

In [371]:
nn_parameters['l0']['W1']

array([1, 1, 1, 1, 0, 0, 1, 1])

In [372]:
ii = np.dot(nn_parameters['l0']['W1'], A)

In [375]:
ii[264]

0

In [373]:
elementos_unicos, contagens = np.unique(ii, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: 0, Contagem: 766
Elemento: 1, Contagem: 30594


In [334]:
ii[ii == 0] = -1

In [335]:
elementos_unicos, contagens = np.unique(ii, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: -1, Contagem: 766
Elemento: 1, Contagem: 30594


In [376]:
ii_img = WOMC.transform_image_forlayer(ii, WOMC.train_size)

In [377]:
elementos_unicos, contagens = np.unique(ii_img, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: -1, Contagem: 13541
Elemento: 1, Contagem: 268699


In [379]:
ii_img.shape

(9, 31360)

In [381]:
ii_img[:,264]

array([ 1,  1,  1,  1, -1,  1,  1,  1,  1])

In [305]:
nn_parameters['l0']

{'W0': array([[ 0., -1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]]),
 'b0': array([[-2],
        [-2],
        [-2],
        [-2],
        [-2],
        [-2],
        [-2],
        [-2]]),
 'W1': array([1, 1, 1, 1, 0, 0, 1, 1]),
 'b1': array([[0.]])}

In [294]:
bias = np.nansum(WW, axis=1) - 1
#bias = np.nansum(WOMC2.W, axis=1) - 1
bias

array([2., 2.])

In [406]:
Wtrain,w_error = WOMC2.window_error_generate_c(W_matrices, WOMC2.train, WOMC2.train_size,WOMC2.ytrain, WOMC2.error_type, 0, 0, bias )

In [407]:
w_error

0.9529606241667963

In [408]:
Wtrain = np.array(Wtrain)
Wtrain.shape

(10, 2, 56, 56)

In [409]:
Wtrain0 = np.array(Wtrain[:, 0, :, :])
Wtrain1 = np.array(Wtrain[:, 1, :, :])

In [301]:
elementos_unicos, contagens = np.unique(Wtrain0, return_counts=True)

# Imprimir os resultados
for elemento, contagem in zip(elementos_unicos, contagens):
    print(f"Elemento: {elemento}, Contagem: {contagem}")

Elemento: -1.0, Contagem: 766
Elemento: 1.0, Contagem: 30594


In [354]:
np.where(Wtrain0[0] == -1)

(array([ 4,  7,  8, 11, 12, 12, 12, 15, 20, 22, 24, 24, 25, 25, 26, 27, 27,
        28, 28, 29, 29, 29, 30, 30, 32, 33, 33, 34, 38, 42, 48, 52, 52, 52,
        52, 52, 52, 52, 53, 54]),
 array([40,  4, 48, 10,  1,  4, 21, 25, 13, 52, 25, 26, 14, 24, 23, 21, 22,
        20, 26, 18, 19, 34, 17, 22, 50,  2, 21, 45, 23,  5, 22, 27, 28, 29,
        30, 31, 32, 33, 13, 15]))

In [355]:
Wtrain0[0][7,4]

-1.0

In [300]:
Wtrain0[0]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [173]:
np.array_equal(Wtrain0, a_img)

True

In [410]:
np.array_equal(Wtrain1, a_img)

True

In [115]:
a_img.shape, a_img.dtype

((10, 56, 56), dtype('int64'))

In [116]:
a_img_float64 = a_img.astype(np.float64)

In [112]:
np_train = np.array(WOMC2.train)

In [114]:
np_train.shape, np_train.dtype

((10, 56, 56), dtype('float64'))

In [56]:
np.array_equal(Wtrain1, a_img)

False

In [632]:
nn_parameters = WOMC.initialize_parameters_nn(WOMC.W)

In [638]:
nn_parameters['l0']['W1'].shape

(1, 32)

In [644]:
np.random.randint(2, size=(1, 32)).shape

(1, 32)

In [457]:
train_b = copy.deepcopy(X2)
ytrain_b = copy.deepcopy(y2)
print(f"X: {train_b.shape}, y: {ytrain_b.shape}")

X: (9, 31360), y: (1, 31360)


In [438]:
AL, caches = WOMC.L_model_forward(train_b, nn_parameters)

In [439]:
cost = WOMC.compute_cost(AL, ytrain_b)
cost

0.9488385468786822

In [440]:
grads = WOMC.L_model_backward(AL, ytrain_b, caches, nn_parameters)

In [417]:
L = len(caches)
dAL = WOMC.calculate_iou_derivative(ytrain_b, AL)

LL = len(caches)-1
dAL.shape

(1, 31360)

In [419]:
for k in reversed(range(WOMC.nlayer)):
    L = (len(nn_parameters['l'+str(k)]) // 2)
    for l in reversed(range(L)):
        print(k,l)

1 1
1 0
0 1
0 0


In [420]:
k=1
l=1
if k == WOMC.nlayer-1 and l == L-1:
    dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dAL, caches[LL])
elif k<WOMC.nlayer-1 and l == L-1:
    dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL], flag = True)
else:
    dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL])
grads["l"+str(k)+"dA" + str(l)] = dA_prev_temp
grads["l"+str(k)+"dW" + str(l)] = dW_temp
grads["l"+str(k)+"db" + str(l)] = db_temp

LL-=1

ValueError: shapes (8,) and (1,31360) not aligned: 8 (dim 0) != 1 (dim 0)

In [421]:
WOMC.L_activation_backward(dAL, caches[LL])

ValueError: shapes (8,) and (1,31360) not aligned: 8 (dim 0) != 1 (dim 0)

In [423]:
linear_cache, activation_cache = caches[LL]

In [424]:
activation_cache.shape

(1, 31360)

In [422]:
dAL.shape

(1, 31360)

In [426]:
dZ = WOMC.tanh_backward(dAL, activation_cache, False)

In [427]:
dA_prev, dW, db = WOMC.L_backward(dZ, linear_cache)

ValueError: shapes (8,) and (1,31360) not aligned: 8 (dim 0) != 1 (dim 0)

In [428]:
dZ.shape

(1, 31360)

In [429]:
A_prev, W, b = linear_cache

In [430]:
m = A_prev.shape[1]
m

31360

In [431]:
dW = np.dot(dZ, A_prev.T) / m

In [432]:
db = np.sum(dZ, axis=1, keepdims=True) / m

In [433]:
dA_prev = np.dot(W.T, dZ)

ValueError: shapes (8,) and (1,31360) not aligned: 8 (dim 0) != 1 (dim 0)

In [434]:
W.shape

(8,)

In [435]:
W

array([1, 1, 1, 1, 0, 0, 1, 1])

In [458]:
parameters, costs = WOMC.nn_model_train(train_b, ytrain_b, nn_parameters, learning_rate=0.01, num_iterations = 2000
                   , print_cost = True )

Cost after iteration 0: 0.9050127976210287
Cost after iteration 10: 0.9138887857303235
Cost after iteration 20: 0.914240147419747
Cost after iteration 30: 0.8972088851925559
Cost after iteration 40: 0.9035465564645488
Cost after iteration 50: 0.9040021961158967
Cost after iteration 60: 0.9008416183844286
Cost after iteration 70: 0.9010373507506776
Cost after iteration 80: 0.9016112372125695
Cost after iteration 90: 0.9067660559017533
Cost after iteration 100: 0.9010373507506776
Cost after iteration 110: 0.9027849250206341
Cost after iteration 120: 0.9060933933332589
Cost after iteration 130: 0.904956349470076
Cost after iteration 140: 0.9016432271465954
Cost after iteration 150: 0.9040559488089357
Cost after iteration 160: 0.9142764805190916
Cost after iteration 170: 0.9016432271465954
Cost after iteration 180: 0.9039263334213574
Cost after iteration 190: 0.9039071542041709
Cost after iteration 200: 0.9132640859010148
Cost after iteration 210: 0.9040510911505336
Cost after iteration 22

In [442]:
parameters

{'l0': {'W0': array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1., 1., 0., 0., 0.]]),
  'b0': array([[-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2],
         [-2]]),
  'W1': array([[1., 1., 1., 1., 0., 0., 1., 1.]]),
  'b1': array([[0.]])},
 'l1': {'W0': array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1., 1., 0., 0., 0.]]),
  'b0': array([[-

In [582]:
v = ytrain_b * probas
np.sum((np.maximum(0, 1 - v)) / probas.size)

0.10746173469387756

In [584]:
union = np.sum(np.maximum(probas,ytrain_b)==1)
interc = np.sum(probas +ytrain_b == 2)
error = (1 - interc/union)
error = error/10
error

0.1

In [593]:
WOMC.hinge_loss_derivative(ytrain_b, probas).shape

(1, 31360)

In [594]:
WOMC.calculate_iou_derivative(ytrain_b, probas).shape

(1, 31360)

In [580]:
WOMC.apply_nn(train_b, ytrain_b, parameters)

1.0

In [591]:
probas, _ = WOMC.L_model_forward(train_b, parameters)

In [586]:
probas = np.where(probas < 0, 0, 1)
probas

array([[0, 0, 0, ..., 0, 0, 0]])

In [587]:
probas = np.where(probas < 0, 0, 1)
y_pred = WOMC.transform_nn_for_image(probas, WOMC.train_size)
y_img = WOMC.transform_nn_for_image(ytrain_b, WOMC.train_size)
n_samples = len(y_img)
for k in range(n_samples):
    y_z = copy.deepcopy(y_img[k])
    y_z[y_z==-1]=0
    union = np.sum(np.maximum(y_pred,y_z)==1)
    interc = np.sum(y_pred +y_z == 2)
    error += (1 - interc/union)
error/n_samples 

In [588]:
n_samples = len(y_img)
for k in range(n_samples):
    y_z = copy.deepcopy(y_img[k])
    y_z[y_z==-1]=0
    union = np.sum(np.maximum(y_pred,y_z)==1)
    interc = np.sum(y_pred +y_z == 2)
    error += (1 - interc/union)
error/n_samples 

1.01

In [569]:
y_pred = WOMC.transform_nn_for_image(probas, WOMC.train_size)

In [570]:
y_pred.shape

(10, 56, 56)

In [577]:
len(y_pred)

10

In [533]:
parameters['l0']['W1']

array([[0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.]])

In [547]:
A, caches = WOMC.L_model_forward(train_b,nn_parameters )#parameters)

In [548]:
A.shape

(1, 31360)

In [556]:
min(min(A)), max(max(A))

(-0.7190001404670499, -0.7085682753071358)

In [558]:
AA = A.copy()

In [560]:
AA[AA<0] = -1
AA[AA>0] = 1

In [561]:
AA = np.where(AA < 0, -1, 1)

In [562]:
AA

array([[-1, -1, -1, ..., -1, -1, -1]])

In [549]:
np.unique(A)

array([-0.71900014, -0.71870236, -0.71862225, -0.71855132, -0.71853079,
       -0.71846606, -0.71845258, -0.71844216, -0.71839648, -0.71832299,
       -0.71830969, -0.71829837, -0.7182587 , -0.71823762, -0.71823096,
       -0.71818222, -0.71814618, -0.71812863, -0.71811871, -0.71810711,
       -0.71810027, -0.71809858, -0.71808837, -0.71807717, -0.71801699,
       -0.71797594, -0.71795559, -0.71791826, -0.71791782, -0.71791003,
       -0.71788775, -0.71788769, -0.7178821 , -0.71785792, -0.71784758,
       -0.7178134 , -0.71780867, -0.71779975, -0.71779145, -0.7177847 ,
       -0.71775142, -0.71774382, -0.71771952, -0.71771324, -0.71769967,
       -0.71767875, -0.7176729 , -0.71765349, -0.71765109, -0.71764682,
       -0.7176361 , -0.71763106, -0.71760294, -0.7176014 , -0.71759447,
       -0.71756047, -0.71755814, -0.7175513 , -0.71754684, -0.71754337,
       -0.71753672, -0.71752957, -0.71752109, -0.71751837, -0.71751799,
       -0.71751751, -0.7175122 , -0.71750326, -0.7175016 , -0.71

In [550]:
nn_parameters['l0']['W1']

array([[-0.09729398, -0.03888766,  0.03597967, -0.11317058, -0.16655322,
        -0.01717888,  0.12285801,  0.00424202,  0.12193508,  0.13514559,
        -0.06911945, -0.01363223,  0.12049994, -0.08342929, -0.06765651,
        -0.04550644, -0.02053767,  0.11377926, -0.07030804,  0.09527589,
        -0.13039654,  0.02502566, -0.14353526, -0.10055253, -0.03480217,
        -0.15719908, -0.04317534,  0.14327622,  0.26272199,  0.03971955,
         0.23795836, -0.270751  ]])

In [ ]:
imagem_shape = (WOMC.img_shape, WOMC.img_shape)
img_reshape = np.reshape(img, (img_size, *imagem_shape))

In [385]:
len(nn_parameters['l'+str(0)]) // 2,len(nn_parameters['l'+str(1)]) // 2

(2, 2)

In [386]:
caches = []

In [387]:
k=0
l=0
A00, cache00 = WOMC.L_activation_forward(
                    train_b, nn_parameters['l'+str(k)]["W" + str(l)], nn_parameters['l'+str(k)]["b" + str(l)])

In [388]:
caches.append(cache00)

In [389]:
A00.shape

(32, 31360)

In [390]:
A_prev = A00
l=1
A01, cache01 = WOMC.L_activation_forward(
                    A_prev, nn_parameters['l'+str(k)]["W" + str(l)], nn_parameters['l'+str(k)]["b" + str(l)])

In [392]:
caches.append(cache01)

In [393]:
A01.shape

(1, 31360)

In [394]:
A_ = WOMC.transform_image_forlayer(A01, 10)
A_.shape

(9, 31360)

In [395]:
A_

array([[-1.        , -1.        , -1.        , ...,  0.03790398,
         0.03790398,  0.03790398],
       [-1.        , -1.        , -1.        , ...,  0.03790398,
         0.03790398,  0.03790398],
       [-1.        , -1.        , -1.        , ...,  0.03790398,
         0.03790398, -1.        ],
       ...,
       [-1.        ,  0.03790398,  0.03790398, ..., -1.        ,
        -1.        , -1.        ],
       [ 0.03790398,  0.03790398,  0.03790398, ..., -1.        ,
        -1.        , -1.        ],
       [ 0.03790398,  0.03790398,  0.03790398, ..., -1.        ,
        -1.        , -1.        ]])

In [396]:
A_[:,0]

array([-1.        , -1.        , -1.        , -1.        ,  0.03790398,
        0.03790398, -1.        ,  0.03790398,  0.03790398])

In [397]:
A

array([[0.03790398, 0.03790398, 0.03790398, ..., 0.03790398, 0.03790398,
        0.03790398]])

In [398]:
k=1
l=0
A_prev = A_
A10, cache10 = WOMC.L_activation_forward(
                    A_prev, nn_parameters['l'+str(k)]["W" + str(l)], nn_parameters['l'+str(k)]["b" + str(l)])

In [399]:
caches.append(cache10)

In [400]:
A10.shape

(32, 31360)

In [401]:
A_prev = A10
l=1
A11, cache11 = WOMC.L_activation_forward(
                    A_prev, nn_parameters['l'+str(k)]["W" + str(l)], nn_parameters['l'+str(k)]["b" + str(l)])

In [423]:
caches.append(cache11)

In [402]:
k<WOMC.nlayer-1

False

In [403]:
A11.shape

(1, 31360)

In [520]:
cost, v = WOMC.compute_cost(A11, ytrain_b)
cost

0.3599063068568191

In [405]:
def hinge_loss_derivative(y, fx):
    condition = 1 - y * fx > 0
    derivative = -y * condition
    return derivative

In [406]:
dAL = hinge_loss_derivative(ytrain_b, A11)
dAL.shape

(1, 31360)

In [424]:
L = len(caches) 
L

4

In [425]:
caches[3]

((array([[-0.97123976, -0.99634593, -0.99634593, ..., -0.99634593,
          -0.99634593, -0.97123976],
         [-0.96660993, -0.99574898, -0.99574898, ..., -0.99993295,
          -0.99993295, -0.9994657 ],
         [-0.96660993, -0.99574898, -0.99574898, ..., -0.99574898,
          -0.99574898, -0.9994657 ],
         ...,
         [-0.99998674, -0.99989434, -0.99989434, ..., -0.99989434,
          -0.99989434, -0.9991581 ],
         [-0.99998674, -0.99989434, -0.99989434, ..., -0.99330887,
          -0.99330887, -0.9991581 ],
         [-0.99998457, -0.99987704, -0.99987704, ..., -0.99987704,
          -0.99987704, -0.99998457]]),
  array([[ 0.14882804,  0.02730311,  0.01761361,  0.05846945,  0.16583191,
          -0.00362824, -0.07882334, -0.11863586,  0.08707894,  0.04683465,
           0.11962379,  0.05704743,  0.03664674, -0.02600384,  0.01059984,
           0.11619086,  0.02175983, -0.00251241, -0.17436242,  0.15310105,
          -0.04713002, -0.03215513,  0.09727703, -0.19055037

In [426]:
nn_parameters

{'l0': {'W0': array([[ 0., -1.,  0., -1., -1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1.,  1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.,  1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1., -1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1., -1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1., -1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1., -1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.,  1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1.,  1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,  1.,  1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.],
         [ 0.,  1.,  0., -1., -1., -1.,  0., -1.,  0.],
         [ 0.,  1.,  0., -1., -1., -

In [409]:
caches[L-1]

((array([[-1.        , -1.        , -1.        , ...,  0.03790398,
           0.03790398,  0.03790398],
         [-1.        , -1.        , -1.        , ...,  0.03790398,
           0.03790398,  0.03790398],
         [-1.        , -1.        , -1.        , ...,  0.03790398,
           0.03790398, -1.        ],
         ...,
         [-1.        ,  0.03790398,  0.03790398, ..., -1.        ,
          -1.        , -1.        ],
         [ 0.03790398,  0.03790398,  0.03790398, ..., -1.        ,
          -1.        , -1.        ],
         [ 0.03790398,  0.03790398,  0.03790398, ..., -1.        ,
          -1.        , -1.        ]]),
  array([[ 0., -1.,  0., -1., -1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1., -1.,  1.,  0.,  1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.,  1., -1.,  0.,  1.,  0.],
         [ 0., -1.,  0.,

In [433]:
current_cache = caches[L - 1]
dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dAL, current_cache)


In [441]:
dA_prev_temp.shape, dW_temp.shape, db_temp.shape

((32, 31360), (1, 32), (1, 1))

In [476]:
LL = len(caches)-1
k=1
L=1
dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dAL, caches[LL])

In [477]:
LL-=1
k=1
L = 0
dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL])

In [478]:
LL-=1
k=0
L = 1
dA_prev_temp2, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL])

ValueError: shapes (32,1) and (9,31360) not aligned: 1 (dim 1) != 9 (dim 0)

In [482]:
linear_cache, activation_cache = caches[LL]

In [502]:
dZ = WOMC.tanh_backward(dA_prev_temp, activation_cache)

In [503]:
dZ.shape

(9, 31360)

In [504]:
middle_row_index = dZ.shape[0] // 2
middle_row_index

4

In [505]:
dZ[middle_row_index].reshape(1, -1).shape

(1, 31360)

In [506]:
dZ = dZ[middle_row_index].reshape(1, -1)

In [507]:
dA_prev, dW, db = WOMC.L_backward(dZ, linear_cache)

In [500]:
A_prev, W, b = linear_cache
m = A_prev.shape[1]
m

31360

In [501]:
dZ.shape, A_prev.T.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [488]:
dW = np.dot(dZ, A_prev.T) / m 

In [489]:
db = np.sum(dZ, axis=1, keepdims=True) / m

In [490]:
W.T.shape, dZ.shape

((32, 1), (9, 31360))

In [510]:
LL = len(caches)-1
grads = {}
for k in reversed(range(WOMC.nlayer)):
    L = (len(nn_parameters['l'+str(k)]) // 2)
    for j in reversed(range(L)):
        if k == WOMC.nlayer-1 and j == L-1:
            dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dAL, caches[LL])
        elif k<WOMC.nlayer-1 and j == L-1:
            dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL], flag = True)
        else:
            dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(dA_prev_temp, caches[LL])
        grads["l"+str(k)+"dA" + str(L)] = dA_prev_temp
        grads["l"+str(k)+"dW" + str(L)] = dW_temp
        grads["l"+str(k)+"db" + str(L)] = db_temp
        
        LL-=1

In [468]:
LL

-1

In [ ]:
nn_parameters['l'+str(k)]["W" + str(L)].shape

In [456]:
#L = len(caches) 

In [461]:
grads = {}
grads["l"+str(k)+"dA" + str(L)] = dA_prev_temp
grads["l"+str(k)+"dW" + str(L)] = dW_temp
grads["l"+str(k)+"db" + str(L)] = db_temp

In [462]:
grads

{'l1dA1': array([[0.07273959, 0.07251333, 0.07251333, ..., 0.07250385, 0.07250385,
         0.07395691],
        [0.01334437, 0.01330286, 0.01330286, ..., 0.01330113, 0.01330113,
         0.01356769],
        [0.00860864, 0.00858186, 0.00858186, ..., 0.00858074, 0.00858074,
         0.00875271],
        ...,
        [0.04307027, 0.0429363 , 0.0429363 , ..., 0.04293069, 0.04293069,
         0.04379106],
        [0.02627867, 0.02619693, 0.02619693, ..., 0.02619351, 0.02619351,
         0.02671845],
        [0.04817691, 0.04802706, 0.04802706, ..., 0.04802078, 0.04802078,
         0.04898317]]),
 'l1dW1': array([[-0.43300747, -0.43306995, -0.43307676, -0.43308165, -0.43305373,
         -0.43307301, -0.43309096, -0.43309267, -0.4330777 , -0.43310356,
         -0.43312438, -0.43311873, -0.43308119, -0.43308072, -0.43314093,
         -0.43315255, -0.43308342, -0.43312427, -0.43311662, -0.43313152,
         -0.43308134, -0.43309455, -0.43310536, -0.4331261 , -0.43310913,
         -0.43312497,

In [432]:
for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = WOMC.L_activation_backward(
        grads["dA" + str(l + 1)], current_cache)
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp

ValueError: operands could not be broadcast together with shapes (9,31360) (32,31360) 

In [354]:
def calculate_iou_derivative(G, P):
    intersection = np.sum(G * P)
    union = np.sum(G) + np.sum(P) - intersection

    # Handle division by zero
    if union == 0:
        return 0.0

    # Calculate the derivative
    derivative = (G * (np.sum(P) + np.sum(G) - 2 * intersection)) / (union ** 2)

    return derivative

# Example usage:
G = np.array([[0, 1, 0],
              [1, 1, 0],
              [0, 0, 0]])

P = np.array([[1, 1, 0],
              [0, 1, 1],
              [0, 0, 0]])

iou_derivative = calculate_iou_derivative(G, P)
print("IoU Derivative:", iou_derivative)

IoU Derivative: [[0.   0.12 0.  ]
 [0.12 0.12 0.  ]
 [0.   0.   0.  ]]


In [210]:
A, cache = WOMC.L_model_forward(train_b, nn_parameters)

In [212]:
A.shape

(9, 31360)

In [67]:
Wtrain.shape

(10, 2, 56, 56)

In [9]:
W_matrices

([array([[ 0., -1.,  0.],
         [-1., -1., -1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1., -1., -1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1., -1.,  1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1., -1.,  1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1.,  1., -1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1.,  1., -1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1.,  1.,  1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [-1.,  1.,  1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [ 1., -1., -1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [ 1., -1., -1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [ 1., -1.,  1.],
         [ 0., -1.,  0.]]),
  array([[ 0., -1.,  0.],
         [ 1., -1.,  1.],
         [ 0.,  1.,  0.]]),
  array([[ 0., -1.,  0.],
         [ 1.,

In [12]:
W_matrices[0]

[array([[ 0., -1.,  0.],
        [-1., -1., -1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1., -1., -1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1., -1.,  1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1., -1.,  1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1.,  1., -1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1.,  1., -1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1.,  1.,  1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [-1.,  1.,  1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 1., -1., -1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 1., -1., -1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 1., -1.,  1.],
        [ 0., -1.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 1., -1.,  1.],
        [ 0.,  1.,  0.]]),
 array([[ 0., -1.,  0.],
        [ 1.,  1., -1.],
        [ 0., -1.,  0.]]),

In [ ]:
w_matricke

In [17]:
weights[0][1]

1

In [15]:
for i, kernel in enumerate(W_matrices[0]):
    print(i)
    print(kernel)
    print('---')

0
[[ 0. -1.  0.]
 [-1. -1. -1.]
 [ 0. -1.  0.]]
---
1
[[ 0. -1.  0.]
 [-1. -1. -1.]
 [ 0.  1.  0.]]
---
2
[[ 0. -1.  0.]
 [-1. -1.  1.]
 [ 0. -1.  0.]]
---
3
[[ 0. -1.  0.]
 [-1. -1.  1.]
 [ 0.  1.  0.]]
---
4
[[ 0. -1.  0.]
 [-1.  1. -1.]
 [ 0. -1.  0.]]
---
5
[[ 0. -1.  0.]
 [-1.  1. -1.]
 [ 0.  1.  0.]]
---
6
[[ 0. -1.  0.]
 [-1.  1.  1.]
 [ 0. -1.  0.]]
---
7
[[ 0. -1.  0.]
 [-1.  1.  1.]
 [ 0.  1.  0.]]
---
8
[[ 0. -1.  0.]
 [ 1. -1. -1.]
 [ 0. -1.  0.]]
---
9
[[ 0. -1.  0.]
 [ 1. -1. -1.]
 [ 0.  1.  0.]]
---
10
[[ 0. -1.  0.]
 [ 1. -1.  1.]
 [ 0. -1.  0.]]
---
11
[[ 0. -1.  0.]
 [ 1. -1.  1.]
 [ 0.  1.  0.]]
---
12
[[ 0. -1.  0.]
 [ 1.  1. -1.]
 [ 0. -1.  0.]]
---
13
[[ 0. -1.  0.]
 [ 1.  1. -1.]
 [ 0.  1.  0.]]
---
14
[[ 0. -1.  0.]
 [ 1.  1.  1.]
 [ 0. -1.  0.]]
---
15
[[ 0. -1.  0.]
 [ 1.  1.  1.]
 [ 0.  1.  0.]]
---
16
[[ 0.  1.  0.]
 [-1. -1. -1.]
 [ 0. -1.  0.]]
---
17
[[ 0.  1.  0.]
 [-1. -1. -1.]
 [ 0.  1.  0.]]
---
18
[[ 0.  1.  0.]
 [-1. -1.  1.]
 [ 0. -1.  0.]]
---
19


In [11]:
np.array(W_matrices[0]) * weights[0][:, np.newaxis, np.newaxis]

array([[[ 0., -0.,  0.],
        [-0., -0., -0.],
        [ 0., -0.,  0.]],

       [[ 0., -1.,  0.],
        [-1., -1., -1.],
        [ 0.,  1.,  0.]],

       [[ 0., -1.,  0.],
        [-1., -1.,  1.],
        [ 0., -1.,  0.]],

       [[ 0., -0.,  0.],
        [-0., -0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0., -0.,  0.],
        [-0.,  0., -0.],
        [ 0., -0.,  0.]],

       [[ 0., -1.,  0.],
        [-1.,  1., -1.],
        [ 0.,  1.,  0.]],

       [[ 0., -1.,  0.],
        [-1.,  1.,  1.],
        [ 0., -1.,  0.]],

       [[ 0., -1.,  0.],
        [-1.,  1.,  1.],
        [ 0.,  1.,  0.]],

       [[ 0., -1.,  0.],
        [ 1., -1., -1.],
        [ 0., -1.,  0.]],

       [[ 0., -0.,  0.],
        [ 0., -0., -0.],
        [ 0.,  0.,  0.]],

       [[ 0., -1.,  0.],
        [ 1., -1.,  1.],
        [ 0., -1.,  0.]],

       [[ 0., -0.,  0.],
        [ 0., -0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0., -0.,  0.],
        [ 0.,  0., -0.],
        [ 0., -0.,  0.]],


In [5]:
bias = np.nansum(WOMC.W, axis=1) - 1

In [68]:
train_b, ytrain_b = WOMC.sort_images(WOMC.train,WOMC.ytrain, WOMC.batch, WOMC.train_size)

In [69]:
Wtrain,w_error =  WOMC.window_error_generate_c(W_matrices, train_b, WOMC.batch,ytrain_b, WOMC.error_type, 0, 0, bias, weights)

In [70]:
w_error

0.8514739141678902

In [73]:
Wtrain2 = np.array([train_b,train_b])

In [85]:
Wtrain2 = np.array([train_b])
Wtrain2 = np.vstack((Wtrain2, np.expand_dims(train_b, axis=0)))


In [86]:
Wtrain2.shape

(2, 10, 56, 56)

In [78]:
Wtrain2 = [train_b]
Wtrain2.append(train_b)


In [79]:
Wtrain2[0].shape

AttributeError: 'list' object has no attribute 'shape'

In [75]:
Wtrain2.shape, Wtrain.shape

((2, 10, 56, 56), (10, 2, 56, 56))

In [72]:
Wtrain[0].shape

(2, 56, 56)

In [34]:
Wtrain[:,-1].shape, len(ytrain_b)

((10, 56, 56), 10)

In [20]:
def mae_loss_gradient(predicted, target):
    return -1 * np.sign(target - predicted) / len(target)

In [37]:
(mae_loss_gradient(Wtrain[:,-1], ytrain_b)* (1 - Wtrain[:,-1]**2)).shape

(10, 56, 56)

In [ ]:
mae_loss_gradient(activated_result2, image[:, :, 0]) * (1 - activated_result2**2) 

In [10]:
def mae_derivative(y_true, y_pred):
    n = len(y_true)
    
    # Calcula o sinal para cada exemplo
    signs = [1 if y_pred[i] > y_true[i] else -1 if y_pred[i] < y_true[i] else 0 for i in range(n)]
    
    # Calcula a derivada parcial para cada exemplo
    derivatives = [1/n * signs[i] for i in range(n)]
    
    return derivatives

In [51]:
X = np.array([[1, 2, 3], [4, 5, 6],[1, 2, 3], [4, 5, 6],[1, 2, 3]])
#X = np.transpose(X)
X.shape

(5, 3)

In [52]:
X

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6],
       [1, 2, 3]])

In [53]:
Y = np.array([0, 1, 0, 1, 1]).reshape(1,-1)
A = np.array([0, 1, 0, 1, 0]).reshape(1,-1)
AL = 1/(1+np.exp(-A))
AL

array([[0.5       , 0.73105858, 0.5       , 0.73105858, 0.5       ]])

In [54]:
W_ = np.array([list(i) for i in itertools.product([-1, 1]
                                                 , repeat=5)])
b_ = np.array([-4]*32).reshape(32,1)

In [55]:
Z = np.dot(W_, X) + b_

In [57]:
W_.shape

(32, 5)

In [58]:
Z.shape

(32, 3)

In [59]:
A = 1/(1+np.exp(-Z))
A.shape

(32, 3)

In [60]:
dAL = - (np.divide(Y, A) - np.divide(1 - Y, 1 - A))
dAL.shape

ValueError: operands could not be broadcast together with shapes (1,5) (32,3) 

In [37]:
Y.shape[1]

6

In [38]:
Y *np.log(AL)+(1.0 - Y) * np.log(1.0 - AL)

array([[-0.69314718, -0.31326169, -0.69314718, -0.31326169, -0.69314718,
        -0.69314718]])

In [23]:
(1.0 - Y) * np.log(1.0 - AL)

/var/folders/zs/n_wh27_s5ln1fk586vbgqnn00000gq/T/ipykernel_24411/352357900.py:1: RuntimeWarning: divide by zero encountered in log
  (1.0 - Y) * np.log(1.0 - AL)
/var/folders/zs/n_wh27_s5ln1fk586vbgqnn00000gq/T/ipykernel_24411/352357900.py:1: RuntimeWarning: invalid value encountered in multiply
  (1.0 - Y) * np.log(1.0 - AL)


array([[ 0., nan,  0., nan,  0.]])

In [28]:

m = Y.shape[1]
cost = -(np.sum(Y * np.log(AL) + (1.0 - Y) * np.log(1.0 - AL))) / m # Logloss error
cost = np.squeeze(cost)
cost


0.5411929833432563

In [29]:
- (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

array([[ 2.        , -1.36787944,  2.        , -1.36787944, -2.        ]])

In [11]:
mae_derivative(ytrain_b, Wtrain[:, -1])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [31]:
mm = mae_loss_gradient(Wtrain[:, -1], ytrain_b)

In [35]:
mm.shape

(10, 56, 56)

In [58]:
W_hood = WOMC.run_window_convolve(train_b, WOMC.batch, W_matrices, 0, 0, bias, weights)

In [14]:
w_error

0.85147391416789

In [ ]:
h_z = h[0][-1]

In [62]:
WOMC.calculate_error(ytrain_b,Wtrain, 'mae')

0.16160714285714287

In [75]:
WOMC.calculate_error(ytrain_b,Wtrain, 'iou')

0.85147391416789

In [63]:
Wtrain_np = np.array(Wtrain)[:, -1]

In [66]:
hz = Wtrain_np[:, -1].copy()
hz[hz==-1]=0

In [74]:
np.abs(hz-ytrain_b).sum()/len(ytrain_b)/ytrain_b[0].size

0.16160714285714287

In [76]:
union = np.sum(np.maximum(hz,ytrain_b)==1)
interc = np.sum(hz +ytrain_b == 2)
error = (1 - interc/union)/len(ytrain_b)
error

0.0853486022229707

In [73]:
ytrain_b[0].size

3136

In [67]:
ytrain_b

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 arr

In [ ]:
hz = Wtr

In [29]:
Wtrain[0][1]

array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
def mae_loss_gradient(predicted, target):
    return -1 * np.sign(target - predicted) / len(target)

In [15]:
dAl = 

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'